In [28]:
import numpy
C = 5
K = 5
def BWT(text):
    """Apply Burrs-Wheeler Transform to input string."""	
    matrix = [text[i:] + text[:i] for i in range(len(text))]
    matrix.sort()
    lastColumn = [r[-1] for r in matrix]
    return "".join(lastColumn)

def countMatrix(last):
    mapping = sorted(set(last))
    symbolDict = {k:v for v, k in enumerate(mapping)}
    count = numpy.zeros(((len(last)+1), len(mapping)), dtype=numpy.int)
    for i in range(len(last)):
        count[i+1] = count[i]
        count[i+1][symbolDict[last[i]]] = count[i][symbolDict[last[i]]] + 1
    count = count.tolist()
    return count

def firstOccurrence(last):
    unilast = sorted(set(last))
    first = sorted(last)
    #print first
    vec = [0] * len(unilast)
    for i,c in enumerate(unilast):
        vec[i] = [j for j,s in enumerate(first) if s == c][0]
    return vec

def BWTMatch(lastColumn,pattern):
    mapping = sorted(set(lastColumn))
    symbolDict = {k:v for v, k in enumerate(mapping)}
    countMat = countMatrix(lastColumn)
    firstOccur = firstOccurrence(lastColumn)

    top = 0
    bottom = len(lastColumn) - 1
    while top <= bottom:
        if len(pattern) > 0:
            symbol = pattern[-1]
            pattern = pattern[:-1]
            if len([r for r in range(top,bottom + 1) if lastColumn[r] == symbol]) > 0:
                top = firstOccur[symbolDict[symbol]] + countMat[top][symbolDict[symbol]]
                bottom = firstOccur[symbolDict[symbol]] + countMat[bottom+1][symbolDict[symbol]] - 1
            else:
                return 0
        else:
            return bottom - top + 1

def multiplePatternMatching(text, patterns):
    """Input that I can use: LastColumn, FirstOccurrence, CheckPointArray, PartialSuffixArray"""
    lastC = BWT(text)
    firstOccur = firstOccurrence(lastC)

    partialsuffixarray = partialSuffixArray(text, K)
    checkpointarray = checkPointArray(lastC, C)
    suffixarray,suffixes = suffixArray(text)

    positions = []
    for pattern in patterns:
        position = PatternMatching(suffixarray,firstOccur,lastC,checkpointarray,partialsuffixarray,pattern)
        if position is not None:
            for pos in position:
                positions.append(pos)
    return positions

def suffixArray(text):
    suffixes = []
    suffixarray = []
    for c in range(len(text)):
        suffixes.append(text[c:])
        suffixarray.append(c)
    suffixarray = [x for (y,x) in sorted(zip(suffixes,suffixarray))]
    suffixes.sort()
    return suffixarray,suffixes

def PatternMatching(suffixarray,firstOccur,lastC,checkpointarray,partialsuffixarray,pattern):
    top,bottom = BetterBWMatching(firstOccur,lastC,checkpointarray,partialsuffixarray,pattern)
    if top is None:
        return None
    position = suffixarray[top:bottom+1]
    return position

def BetterBWMatching(firstOccur,lastC,checkpointarray,partialsuffixarray,pattern):
    top = 0
    bottom = len(lastC)-1

    while top<=bottom:
        if len(pattern) > 0:
            symbol = pattern[-1]
            pattern = pattern[:-1]
            if lastC[top:bottom+1].count(symbol) > 0:
                top = firstOccur['$ACGT'.index(symbol)] + countMatrix1(lastC,checkpointarray,top,symbol)
                bottom = firstOccur['$ACGT'.index(symbol)] + countMatrix1(lastC,checkpointarray,bottom+1,symbol) - 1
            else:
                return None,None
        else:
            return top,bottom
def suffixArray(text):
    suffixes = []
    suffixarray = []
    for c in range(len(text)):
        suffixes.append(text[c:])
        suffixarray.append(c)
    suffixarray = [x for (y,x) in sorted(zip(suffixes,suffixarray))]
    return suffixarray, suffixes

def partialSuffixArray(text,K):
    suffixarray = suffixArray(text)
    partialsuffixarray = {}
    for i, s in enumerate(suffixarray):
        if s % K ==0:
            partialsuffix = i
            partialsuffixarray[i] = s
    return partialsuffixarray

def countMatrix1(lastC,checkpointarray,idx,symbol):
    '''implement countMat[index]['ACGT'.index(symbol)] based on checkpointarray'''
    idxStart = (idx/C)*C
    rank = checkpointarray[idxStart]['$ACGT'.index(symbol)]
    if lastC[idxStart:idx].count(symbol)>0:
        count = lastC[idxStart:idx].count(symbol)
        rank = rank + count
    return rank

def checkPointArray(last, C):
    countmatrix = countMatrix(last)
    checkpointarray = {}
    for i, c in enumerate(countmatrix):
        if i % C == 0 :
            checkpointarray[i] = c
    return checkpointarray

def hamming_distance(str1,str2):
    hd = 0
    minlen = min(len(str1),len(str2))
    maxlen = max(len(str1),len(str2))
    for i in range(minlen):
        if str1[i] != str2[i]:
            hd += 1
    hd += maxlen-minlen
    return hd

def multipleApproximatePatternMatching(text,patterns,d):
    lastC = BWT(text)
    firstOccur = firstOccurrence(lastC)

    partialsuffixarray = partialSuffixArray(text, K)
    checkpointarray = checkPointArray(lastC, C)
    suffixarray,suffixes = suffixArray(text)

    positions = []
    for pattern in patterns:
        '''seed preparation:divide pattern into d + 1 substring, with the length floor(n/(d+1))'''
        n = len(pattern)
        k = n / (d+1)
        #print "pattern",pattern,"n",n,"k",k,"d",d
        # for each pattern, if we hits same position in the text multiple times, we only keep one record.
        posstarts = []
        for i in range(d+1):
            if i == d:
                pat = pattern[i*k:]
            else:
                pat = pattern[i*k:(i+1)*k]
            '''seed detection: for each seed (pat), do exact pattern matching to find which seeds match Text exactly'''
            position = PatternMatching(suffixarray,firstOccur,lastC,checkpointarray,partialsuffixarray,pat)

            if position is not None:
                for pos in position:
                    '''seed extension: extend seeds in both directions to verify whether Pattern occurs in Text with at most d mismatches.'''
                    posstart = pos-k*i
                    dist = hamming_distance(text[posstart:posstart+n],pattern)
                    if dist is not None and dist <= d:
                        posstarts.append(posstart)
        positions.extend(list(set(posstarts)))
    return positions

In [29]:
with open('MultipleApproximatePatternMatching/inputs/input_1.txt') as f:
    text = f.readline()
    pattern_list = f.readline().strip().split(' ')
    d = int(f.readline())
    ans = multipleApproximatePatternMatching(text, pattern_list, d)
    print(ans)

TypeError: unsupported operand type(s) for %: 'list' and 'int'

In [ ]:
print(ans)